# Models

### Data Processing

In [1]:
import pandas as pd
import numpy as np

Choose top 4 features (chosen by PCA Analysis)

In [2]:
data = pd.read_csv('../normalized_data.csv')
data = data[['Social support', 'Log GDP per capita', 'Healthy life expectancy at birth', 'Freedom to make life choices', 'Happiness_Score_Percentile']]
data = data.dropna()
data.iloc[0]

Social support                     -2.888644
Log GDP per capita                 -1.565348
Healthy life expectancy at birth   -1.697393
Freedom to make life choices       -1.111509
Happiness_Score_Percentile          8.000000
Name: 0, dtype: float64

Split into testing and training

In [3]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2)

y_train = train['Happiness_Score_Percentile']
x_train = train.drop(['Happiness_Score_Percentile'], axis=1)
y_test = test['Happiness_Score_Percentile']
x_test = test.drop(['Happiness_Score_Percentile'], axis=1)

assert len(x_train) == len(y_train)
assert len(x_test) == len(y_test)
assert len(x_train.iloc[0]) == len(x_test.iloc[0])
print(len(x_train), 'train examples')
print(len(x_test), 'test examples')

931 train examples
233 test examples


### Models

Regular Linear Regression

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score

reg = LinearRegression().fit(x_train, y_train)
y_pred = reg.predict(x_test)
print(reg.score(x_test, y_test))

#Find accuracy and error on test dataset
print("RMS: ", (mean_squared_error(y_test, y_pred))**0.5)
print("Accuracy: ", accuracy_score(y_test, y_pred.round()))

0.7259120153746494
RMS:  1.1625109157881646
Accuracy:  0.33476394849785407


Linear Regression with L1 and L2 Regularization and Cross Validation selection

In [13]:
from sklearn.linear_model import ElasticNetCV

regr = ElasticNetCV()
regr.fit(x_train, y_train)
print(regr.score(x_test, y_test))

y_pred = regr.predict(x_test)

# print(y_pred)
print("RMS: ", (mean_squared_error(y_test, y_pred))**0.5)
#Find accuracy
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_test, y_pred.round()))
# print("Accuracy: ", accuracy_score(y_test, y_pred))


0.7251769810221109
RMS:  1.1640686516154164
Accuracy:  0.33047210300429186


Ordinal Regression

In [14]:
! pip install statsmodels

In [15]:
from statsmodels.miscmodels.ordinal_model import OrderedModel

mod_prob = OrderedModel(y_train, x_train, distr='logit')
res_prob = mod_prob.fit(method='bfgs')
res_prob.summary()

Optimization terminated successfully.
         Current function value: 1.553781
         Iterations: 36
         Function evaluations: 37
         Gradient evaluations: 37


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OrderedModel Results                                 
======================================================================================
Dep. Variable:     Happiness_Score_Percentile   Log-Likelihood:                -1446.6
Model:                           OrderedModel   AIC:                             2919.
Method:                    Maximum Likelihood   BIC:                             2982.
Date:                        Mon, 28 Nov 2022                                         
Time:                                20:23:14                                         
No. Observations:                         931                                         
Df Residuals:                             918                                         
Df Model:                                  13                                         
====================================================================================================
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Social support                      -0.9322      0.096     -9.692      0.000      -1.121      -0.744
Log GDP per capita                  -1.1390      0.129     -8.796      0.000      -1.393      -0.885
Healthy life expectancy at birth    -0.7138      0.120     -5.933      0.000      -0.950      -0.478
Freedom to make life choices        -0.7454      0.071    -10.549      0.000      -0.884      -0.607
0/1                                 -4.7433      0.180    -26.417      0.000      -5.095      -4.391
1/2                                  0.2279      0.106      2.155      0.031       0.021       0.435
2/3                                  0.2248      0.088      2.541      0.011       0.051       0.398
3/4                                  0.4896      0.069      7.112      0.000       0.355       0.624
4/5                                  0.3287      0.078      4.217      0.000       0.176       0.481
5/6                                  0.4818      0.077      6.289      0.000       0.332       0.632
6/7                                  0.4983      0.082      6.052      0.000       0.337       0.660
7/8                                  0.1982      0.119      1.665      0.096      -0.035       0.432
8/9                                  0.7176      0.148      4.862      0.000       0.428       1.007
====================================================================================================
"""

In [16]:
y_pred = res_prob.model.predict(res_prob.params, x_test)
y_pred = np.argmax(y_pred, axis=1)

print("RMS: ", (mean_squared_error(y_test, y_pred))**0.5)
print("Accuracy: ", accuracy_score(y_test, y_pred))

RMS:  1.1990697395661971
Accuracy:  0.38626609442060084


c:\Users\Rohit\Anaconda3\envs\cs7643-a2\lib\site-packages\statsmodels\miscmodels\ordinal_model.py:419: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  xb = xb[:, None]


Decision Tree

In [21]:
#decision tree and visualization
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import graphviz

# clf = DecisionTreeClassifier(random_state=0, max_depth=10)
# clf = clf.fit(x_train, y_train)
# y_pred = clf.predict(x_test)
# print(clf.score(x_test, y_test))

# #visualize tree
# dot_data = tree.export_graphviz(clf, out_file=None)
# graph = graphviz.Source(dot_data)
# graph.render("happiness_depth10")

depths = [3,4,5,6,7,8,9,10]

for depth in depths:
    clf = DecisionTreeClassifier(random_state=0, max_depth=depth)
    clf = clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print("Depth: ", depth, " Accuracy: ", clf.score(x_test, y_test))

    dot_data = tree.export_graphviz(clf, out_file=None)
    graph = graphviz.Source(dot_data)
    graph.render("tree_visualizations/happiness_depth" + str(depth))



Depth:  3  Accuracy:  0.39914163090128757
Depth:  4  Accuracy:  0.40772532188841204
Depth:  5  Accuracy:  0.38626609442060084
Depth:  6  Accuracy:  0.4334763948497854
Depth:  7  Accuracy:  0.41201716738197425
Depth:  8  Accuracy:  0.3776824034334764
Depth:  9  Accuracy:  0.38626609442060084
Depth:  10  Accuracy:  0.4034334763948498
